<a href="https://colab.research.google.com/github/duyvm/funny_stuff_with_llm/blob/main/learning-rag/Langchain_multi_vectors_per_document.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Option 1: Mount Google Drive for external libraries

        from google.colab import drive
        import sys
        drive.mount('/content/gdrive')
        sys.path.append('/content/gdrive/My Drive/External Libraries')

# Option 2: Install libraries

In [ ]:
%pip install --quiet --upgrade langchain-chroma langchain[openai] langchain langchain-community langgraph langchain-core langchain-text-splitters> /dev/null

In [ ]:
from google.colab import userdata
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = f"langchain-learning-rag"
os.environ["LANGSMITH_API_KEY"] = userdata.get('LANGSMITH_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Overview

Guide: [How to retrieve using multiple vectors per document](https://python.langchain.com/docs/how_to/multi_vector/)

These techniques are to improve the accuracy of RAG by enhancing the accuracy of embedded documents retrieval. To achieve the high accuracy, they carefully creates multi-vectors for smaller chunks of original document. When the search hits one of those chunks, instead of return the chunk, we return the larger document.

**Detail methods**

1. Smaller chunk: Split document into smaller chunks and embed these chunks (same as basic RAG). Each chunk has a metadata that link to where original document is stored. Then the system will retrieve the original document and feed it to the llm for answer systhesis.

2. Summary: create a summary for document and embed it along (or instead of) the document. Also link it with original document by metadata

3. Hypothetical questions: create hypothetical questions that each document would be approriate to answer, embed these questions with (or instead of) original documents. This is also a method called manually adding embeddings, giving more control over the retrieval process by explicitly adding questions or queries that lead to the documents.

# Load data

1. Download sample document files from github

2. Load original documents into `langchain_core.documents.base.Document` type

3. Observe the `Documents` data (content, metadata...)

4. Split them into smaller `Documents` with specific length (no overlap)

In [ ]:
import requests
from os import getcwd

# load files from github
file_names = [
    "paul_graham_essay.txt",
    "state_of_the_union.txt",
]

raw_base_url = "https://raw.githubusercontent.com/duyvm/funny_stuff_with_llm/refs/heads/main/learning-rag/docs/{file_name}"

for filename in file_names:
    url = raw_base_url.format(file_name=filename)
    r = requests.get(url)

    f = open(filename,'wb')
    f.write(r.content)


In [ ]:
from langchain.storage import InMemoryByteStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader

loaders = [
    TextLoader("paul_graham_essay.txt"),
    TextLoader("state_of_the_union.txt")
]

docs = []
for loader in loaders:
    docs.extend(loader.load())

# inspect the document type
print(docs[0].__dict__.keys())
print(docs[0].type)
print(docs[0].metadata)
print(len(docs[0].page_content))
print(docs[0].page_content[:1000])

dict_keys(['id', 'metadata', 'page_content', 'type'])
Document
{'source': 'paul_graham_essay.txt'}
75012
What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.

The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.

The language

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000)

docs = text_splitter.split_documents(docs)

# inspect splitted document
print(len(docs))
print(docs[0].__dict__.keys())
print(docs[0].type)
print(docs[0].metadata)
print(len(docs[0].page_content))
print(docs[0].page_content[:1000])

12
dict_keys(['id', 'metadata', 'page_content', 'type'])
Document
{'source': 'paul_graham_essay.txt'}
9728
What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.

The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.

The langua

In [ ]:
# check overlapping
print(docs[0].page_content[-100:])
print(docs[1].page_content[:100])

tist you could be truly independent. You wouldn't have a boss, or even need to get research funding.
I had always liked looking at paintings. Could I make them? I had no idea. I'd never imagined it was


In [ ]:
# index the chunk Chroma
vector_store = Chroma(collection_name="full_documents", embedding_function=OpenAIEmbeddings())

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


# Smaller chunks

- Smaller chunks for embedding but larger chunks for information context

- Allow capturing semantic as closely as possible but as much context as possible for answer synthesis

- There are two type of documents:

  - The parent documents: hold the full context

  - The child documents: smaller chunks splitted from the parent document

- Use two `vector_store` to seperate two types of document above and link them with an identifier

In [ ]:
import uuid
from langchain.retrievers.multi_vector import MultiVectorRetriever

ID_KEY_NAME = "doc_id"
store = InMemoryByteStore()

multi_vector_retriever = MultiVectorRetriever(
    vectorstore=vector_store,
    byte_store=store,
    id_key=ID_KEY_NAME,
)

# create id for docs
docs_id = [str(uuid.uuid4()) for _ in docs]

In [ ]:
docs_id

['b9f16a93-7f46-4c07-aae4-5cc4692829b2',
 'd7194779-5539-492a-871b-6cfb3371585d',
 'e46a3316-61dd-445e-96f0-89957c6f65a3',
 '460b1460-40bd-46c6-ae36-f1bc6e45802e',
 '8e8a2e2d-6a4b-49e3-9a02-de8e093c1632',
 '456586cb-d8d4-47c6-ac07-39a34882a601',
 'b4781a53-6f3d-4e4b-8690-fde45a12f081',
 '35181097-87cf-4644-bb3e-659ba92dfcf9',
 '098563cb-ee50-448f-85ca-0d65d1b6b59b',
 'ed0f4c9f-bcdf-45a6-8c31-56ed72427132',
 '579974fa-240c-469b-beff-08429e17721d',
 '287123ea-6f70-4866-a74b-be2a76a852c4']

In [ ]:
# generate child document by splitting parent document
# store the identifier in metadata of document
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
child_docs = []

for doc, doc_id in zip(docs, docs_id):
    _child_docs = child_text_splitter.split_documents([doc])
    for _child_doc in _child_docs:
        _child_doc.metadata[ID_KEY_NAME] = doc_id
    child_docs.extend(_child_docs)

# inspec 1st and 2nd child docs
print(child_docs[0].metadata)
print(child_docs[1].metadata)

{'source': 'paul_graham_essay.txt', 'doc_id': 'b9f16a93-7f46-4c07-aae4-5cc4692829b2'}
{'source': 'paul_graham_essay.txt', 'doc_id': 'b9f16a93-7f46-4c07-aae4-5cc4692829b2'}


In [ ]:
# compare content with parent doc
print("--- Child docs ---")
print(child_docs[0].page_content[:100])
print(child_docs[1].page_content[:100])

print("--- Parent docs ---")
print(docs[0].page_content[:100])
print(docs[0].page_content[400:500])

--- Child docs ---
What I Worked On

February 2021

Before college the two main things I worked on, outside of school, 
The first programs I tried writing were on the IBM 1401 that our school district used for what was t
--- Parent docs ---
What I Worked On

February 2021

Before college the two main things I worked on, outside of school, 
writing were on the IBM 1401 that our school district used for what was then called "data processing


In [ ]:
# add child documents to vectorstore for embedding
# add id and documents to docstore for reference
multi_vector_retriever.vectorstore.add_documents(child_docs)
multi_vector_retriever.docstore.mset(list(zip(docs_id, docs)))

In [ ]:
# test it
doc = multi_vector_retriever.vectorstore.similarity_search("justice breyer")[0]
print(doc)
print(len(doc.page_content)) # checking child or parent

page_content='Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court.' metadata={'source': 'state_of_the_union.txt', 'doc_id': '579974fa-240c-469b-beff-08429e17721d'}
390


In [ ]:
# get the parent doc
doc = multi_vector_retriever.invoke("justice breyer")[0]

print(doc)
print("-"*20)
print(len(doc.page_content)) # checking child or parent

page_content='But in my administration, the watchdogs have been welcomed back. 

We’re going after the criminals who stole billions in relief money meant for small businesses and millions of Americans.  

And tonight, I’m announcing that the Justice Department will name a chief prosecutor for pandemic fraud. 

By the end of this year, the deficit will be down to less than half what it was before I took office.  

The only president ever to cut the deficit by more than one trillion dollars in a single year. 

Lowering your costs also means demanding more competition. 

I’m a capitalist, but capitalism without competition isn’t capitalism. 

It’s exploitation—and it drives up prices. 

When corporations don’t have to compete, their profits go up, your prices go up, and small businesses and family farmers and ranchers go under. 

We see it happening with ocean carriers moving goods in and out of America. 

During the pandemic, these foreign-owned companies raised prices by as much as 1,00

# Summary

- Provide summary for each chunk, may lead to better accuracy and retrieval

In [ ]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("openai:gpt-4o-mini")

In [ ]:
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("Summarize the following document:\n{doc}")
    | llm
    | StrOutputParser()
)

In [ ]:
summaries = chain.batch(docs, {"max_concurrency": 5})

In [ ]:
# inspect the sumaries
print(len(summaries))
print(summaries[0])

12
The document outlines the author's journey through early writing and programming experiences from childhood to graduate school. It begins with their initial attempts at writing short stories and programming on an IBM 1401 in junior high. The author describes a turning point with the advent of microcomputers, which made programming more accessible and interactive. After acquiring a TRS-80 and developing simple programs, the author initially planned to study philosophy in college, believing it to explore ultimate truths. However, upon discovering the limitations of philosophy courses, they switched to studying artificial intelligence (AI), influenced by works of fiction and documentaries that fueled their passion for intelligent systems.

In graduate school, the author came to a realization that contemporary AI, particularly natural language processing, was fundamentally flawed and did not equate to genuine understanding. This led to a shift in focus toward Lisp programming, culminati

In [ ]:
import uuid
from langchain.retrievers.multi_vector import MultiVectorRetriever

# create storage, same as above
ID_KEY_NAME = "doc_id"
vector_store = Chroma(collection_name="summaries", embedding_function=OpenAIEmbeddings())
store = InMemoryByteStore()

multi_vector_retriever = MultiVectorRetriever(
    vectorstore=vector_store,
    byte_store=store,
    id_key=ID_KEY_NAME,
)

doc_ids = [str(uuid.uuid4()) for _ in summaries]

summary_docs = [
    Document(page_content=summary, metadata={ID_KEY_NAME: doc_ids[i]})
    for i, summary in enumerate(summaries)
]

multi_vector_retriever.vectorstore.add_documents(summary_docs)
multi_vector_retriever.docstore.mset(list(zip(doc_ids, docs)))

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [ ]:
sub_doc = multi_vector_retriever.vectorstore.similarity_search("justice breyer")[0]
print(sub_doc.page_content)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


In a recent address, the President emphasized the significant constitutional duty of nominating a Supreme Court Justice, having nominated Judge Ketanji Brown Jackson, a highly regarded legal mind, to continue Justice Breyer’s legacy. The President highlighted the importance of securing the border and reforming the immigration system, advocating for measures that use technology to combat drug smuggling, expedite asylum cases, and provide pathways to citizenship for vulnerable groups. He called for a comprehensive approach to immigration reform that has broad support across various sectors.

The President also focused on advancing liberty and justice by protecting women's rights and access to healthcare, particularly in light of ongoing challenges to Roe v. Wade. He pledged support for LGBTQ+ rights through the proposed bipartisan Equality Act. Emphasizing the need for unity, he presented a "Unity Agenda" consisting of four key issues: combating the opioid epidemic, addressing mental hea

In [ ]:
retrieved_doc = multi_vector_retriever.invoke("justice breyer")[0]
print(len(retrieved_doc.page_content))
print("\n")
print(retrieved_doc.page_content)

9194


One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence. 

A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. 

And if we are to advance liberty and justice, we need to secure the Border and fix the immigration system. 

We can do both. At our border, we’ve installed new technology like cutting-edge scanners to better detect drug smuggling.  

We’ve set up joint patrols with Mexico and Guatemala to catch more human traffickers.  

We’re pu

# Hypothetical Queries

- Use llm to generate list of hypothetical questions that could be used to ask of the target document

- These hypothetical queries might bear semantic similarity close to the queries will be asked in RAG

- These hypothetical queries can be embedded and associated with asked document to improve retrieval

In [ ]:
from typing import List
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

class HypotheticalQuestions(BaseModel):
    """Generate hypothetical questions."""
    questions: List[str] = Field(..., description="List of hypothetical questions")

chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("Generate a list of exaclty 3 hypothetical questions that the below document can be used to answer:\n\n{doc}") # adjust number of questions
    | ChatOpenAI(max_retries=0, model="gpt-4o-mini").with_structured_output(HypotheticalQuestions)
    | (lambda x: x.questions)
)

In [ ]:
# generate questions for first doc
chain.invoke(docs[0])

['If you had the opportunity to take a different path in your college studies, how might a focus on computer programming have changed your career trajectory?',
 'What do you think would have happened to the development of AI if you had not realized its limitations during graduate school?',
 'How might your experience with the IBM 1401 or the TRS-80 influence your attitude towards modern programming languages and their ease of use?']

In [ ]:
# generate all questions and embed them
hypothetical_questions = chain.batch(docs, {"max_concurrency": 5})

vector_store = Chroma(collection_name="hypothetical_questions", embedding_function=OpenAIEmbeddings())

multi_vector_retriever = MultiVectorRetriever(
    vectorstore=vector_store,
     byte_store=InMemoryByteStore(),
    id_key=ID_KEY_NAME,
)

doc_ids = [str(uuid.uuid4()) for _ in docs]

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [ ]:
# inspect the batch of questions
print(len(hypothetical_questions))
print(len(hypothetical_questions[0]))
print(hypothetical_questions[0])

12
3
['If you had decided to continue pursuing a career in philosophy, how do you think your views on AI and technology would have been different?', 'What impact do you think the early experience with the IBM 1401 had on your later decision to focus on programming and artificial intelligence?', 'How might your career path have changed if you had been accepted into one of the other graduate schools you applied to, such as MIT or Yale?']


In [ ]:
question_docs = []

for i, question_list in enumerate(hypothetical_questions):
    question_docs.extend([
        Document(page_content=question, metadata={ID_KEY_NAME: doc_ids[i]})
        for question in question_list
    ])

multi_vector_retriever.vectorstore.add_documents(question_docs)
multi_vector_retriever.docstore.mset(list(zip(doc_ids, docs)))

In [ ]:
# testing
sub_docs = multi_vector_retriever.vectorstore.similarity_search("Y Combinator instead of Sam Altman")

for _doc in sub_docs:
    print(_doc.page_content)
    print("-"*20)

How could Robert Morris's career have been different if he had decided to take over Y Combinator instead of Sam Altman?
--------------------
What would have happened if the founders of Y Combinator had decided to follow the traditional venture capital model instead of creating their own investment firm?
--------------------
What might have happened had Rtm not given advice regarding Y Combinator?
--------------------
What if Paul Graham had chosen to pursue a different passion instead of painting after stepping away from Y Combinator?
--------------------


In [ ]:
retrieved_docs = multi_vector_retriever.invoke("justice breyer")
len(retrieved_docs[0].page_content)

9194

# Parent Document Retriever

- Extended class of `MultiVectorRetriever`

- Special crafted for handling parent-child documents retrieval easily

In [ ]:
from langchain.retrievers import ParentDocumentRetriever

## Retrieve full document using ParentDocumentRetriever

In [ ]:
# set splitter for child chunks
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

vector_store = Chroma(collection_name="parent_document_retriever", embedding_function=OpenAIEmbeddings())

parent_retriever = ParentDocumentRetriever(
    vectorstore=vector_store,
    docstore=InMemoryByteStore(),
    child_splitter=child_splitter,
)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [ ]:
# ParentDocumentRetriever automatically add documents
# to vectorstore and docstore
parent_retriever.add_documents(docs, ids=None)

# check document ids
list(parent_retriever.docstore.yield_keys())

['94003fc7-2dc9-496c-827c-2d53174a4c28',
 'e0b79d1c-7965-445c-ae78-eb1da03565a9']

In [ ]:
sub_docs = parent_retriever.vectorstore.similarity_search("justice breyer")
# inspect number of retrieved child chunks
print(len(sub_docs))
print()
# inspect size of chilk chunk
print(len(sub_docs[0].page_content))
print()
# print out a child chunk content
print(sub_docs[0].page_content)

4

390

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court.


In [ ]:
# get the parent document
retrieved_parent_doc = parent_retriever.invoke("justice breyer")[0]
print(len(retrieved_parent_doc.page_content))
print("\n")
# inspect first 100 chars
print(retrieved_parent_doc.page_content[:100])

38540


Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and th


## Retrieve larger chunk with ParentDocumentRetriever

In [ ]:
# prepare splitter for both child chunks and parent chunks
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

vector_store = Chroma(collection_name="parent_document_retriever_2", embedding_function=OpenAIEmbeddings())

parent_retriever = ParentDocumentRetriever(
    vectorstore=vector_store,
    docstore=InMemoryByteStore(),
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [ ]:
# add documents, auto splitting both child and parent chunks
parent_retriever.add_documents(docs)

# inspect number of parent chunks and their key
print(len(list(parent_retriever.docstore.yield_keys())))
print(list(parent_retriever.docstore.yield_keys())[:20])

66
['35f72eef-67c1-449a-9191-6ca2f829407b', '1339a08a-a890-4940-9e04-ff1253e105cd', '589f5b0d-32bd-4c20-9424-57a2cc7609e0', '7fcb8d58-3271-432d-ab38-02adf2a8a991', '372f6487-3e32-49fc-82e4-d24930a0d195', 'c4787341-f5a5-4e16-974e-8ad437da6b53', '7b1deea5-8550-49c0-b321-1b27ea65b903', '9bb7fca9-3c7c-449c-aa23-f6466fb28cfa', '5a60c420-6249-428e-b39a-f1e84bc414f7', '8804b27b-3032-4bc7-8eb3-5e2d6f2267b3', '6c24bb53-22cb-4474-a2d4-0020d32e48a8', 'b93f3e22-0669-4336-8261-10fe8a01f2ef', '71ffe649-18d2-40e5-aa23-8f95dc47f8d6', 'de744a7b-eeaf-4e84-86bd-3551b142d581', 'a046798a-bfbb-41c0-a47a-3d928a6c6a06', 'cdcd21a0-748d-4e62-ae9f-0846419b6f46', '23c06b42-be7a-40e4-b4e3-524287c5907d', '8437b357-a2a9-4730-9340-b30082004ab2', 'c9482862-a885-4239-a8af-43c98077c9f7', 'c9dfffda-ba9a-482e-a5cc-5d1da992c285']


In [ ]:
# inspect parent chunk
retrieved_parent_docs = parent_retriever.invoke("justice breyer")
print(len(retrieved_parent_docs))
print("\n")
print(len(retrieved_parent_docs[0].page_content))
print("\n")
print(retrieved_parent_docs[0].page_content)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


1


1849


In state after state, new laws have been passed, not only to suppress the vote, but to subvert entire elections. 

We cannot let this happen. 

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence. 

A former top litigator in private practice. A former fe